# IFN711 - Budget Analysis for Leap in!

### The following cell install the required dependencies. 

In [ ]:
!pip install -r requirements.txt

## Import statements

In [62]:
import pandas as pd
import pymysql
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori
# Imports for the neural network
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.linear_model import LogisticRegression
# Load the .env file for database credentials
from dotenv import load_dotenv

## Sign into the Database using credentials either stored in .env file, or **manual input**.

In [63]:
# Declare variables
host = None
port = None
db = None
user = None
password = None
# Load the .env file to access database credentials
if load_dotenv():
    # Assign required values to variables
    host=os.getenv("host")
    port=int(os.getenv("port"))
    db=os.getenv("dbname")
    user=os.getenv("user")
    password=os.getenv("password")
    # Confirm loading of credentials
    print("Database Credentials Loaded Successfully.")
# If unable to load env file, take manual input.
else:
    # Confirm failure of loading of credentials
    print("Unable to detect Database Credentials. Please enter credentials manually.\n")
    # Request manual entry of credentials
    host = input("\nPlease enter host address: ")
    user = input("\nPlease enter username: ")
    password = input("\nPlease enter password: ")
    port = input("\nPlease enter port number: ")
    db = input("\nPlease enter database name: ")
    # Convert port to int
    port = int(port)
####### Connection to Client Database #######
conn = pymysql.connect(host=host, user=user, port=port, password=password, db=db)
# Print connection confirmation
print(conn)

Database Credentials Loaded Successfully.


## Make a dataframe of customer/user information.

In [64]:
df_user= pd.read_sql("select m.id as member_id, m.membership_number, m.status, m.price_zone_code, m.member_key, s.u_ndis_number, s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member m  on s.u_ndis_number = m.membership_number left join libe_leapinprod_memberregion r on r.MemberId = s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null;", con=conn)

# Display summary information of df_user          
df_user.info()
df_user.head(100)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   member_id          6115 non-null   float64
 1   membership_number  6115 non-null   object 
 2   status             6115 non-null   object 
 3   price_zone_code    6115 non-null   object 
 4   member_key         6115 non-null   object 
 5   u_ndis_number      6241 non-null   object 
 6   u_disabilities     6241 non-null   object 
 7   u_gender           6241 non-null   object 
 8   u_date_of_birth    6239 non-null   object 
 9   SA1                6021 non-null   float64
 10  SA2                6021 non-null   float64
 11  SA3                6021 non-null   float64
 12  SA4                6021 non-null   float64
dtypes: float64(5), object(8)
memory usage: 634.0+ KB


,member_id,membership_number,status,price_zone_code,member_key,u_ndis_number,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4
0,1613.0,916644409,managed,ACT_NSW_QLD_VIC,d74bcd00-d205-11ea-b0d9-53acaecd0b82,916644409,,,1922-11-30,3100614.0,31006.0,30101.0,301.0
1,3132.0,262354568,managed,ACT_NSW_QLD_VIC,5d057910-dce6-11ea-812e-09cdf4382371,262354568,,,2007-04-06,1144614.0,11446.0,12303.0,123.0
2,1786.0,233367363,managed,ACT_NSW_QLD_VIC,222759f0-d212-11ea-8b42-db603b1b9020,233367363,,Female,1954-03-26,0.0,0.0,0.0,0.0
3,2562.0,981297214,managed,ACT_NSW_QLD_VIC,b0a9ca70-d50d-11ea-8224-836b70d82925,981297214,,Female,1978-12-24,3138504.0,31385.0,31401.0,314.0
4,1614.0,631304335,managed,ACT_NSW_QLD_VIC,d75087f0-d205-11ea-b0d9-53acaecd0b82,631304335,,,1995-08-18,2142119.0,21421.0,21701.0,217.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2792.0,266947556,managed,ACT_NSW_QLD_VIC,c2ce1270-d6d3-11ea-9cd1-f194648955a5,266947556,"Hypothyroidism , Fibromyalgia , Thyroid , Deaf",Female,1932-04-07,1158802.0,11588.0,12504.0,125.0
96,1964.0,723367568,managed,ACT_NSW_QLD_VIC,614c9420-d21b-11ea-b617-61a96cd5c136,723367568,,Male,1978-09-21,0.0,0.0,0.0,0.0
97,551.0,708296665,managed,ACT_NSW_QLD_VIC,c3e4aad0-c7e6-11ea-bd70-2bdba120f26c,708296665,,,2001-09-15,2110528.0,21105.0,20601.0,206.0
98,2079.0,331401992,managed,ACT_NSW_QLD_VIC,a80f41a0-d220-11ea-b70a-d17e589b1829,331401992,,Male,1940-04-30,3107107.0,31071.0,30304.0,303.0


In [ ]:
# df_user['u_disabilities'].value_counts()

## Make a dataframe of claims information.

In [ ]:
df_claims= pd.read_sql("SELECT c.id as claim_id, c.invoice_id, c.state, c.risk_level, c.start_date FROM HH_claim c;", con=conn)

# Display summary information of df_user
df_claims.info()
df_claims.head(100)


 ## Make a dataframe of Invoice information.

In [ ]:
df_invoices= pd.read_sql("SELECT i.id as invoice_id, i.member_id, i.invoice_total, i.funded_total, i.funded_date FROM HH_invoice i;", con=conn)

# Display summary information of df_invoices          
df_invoices.info()
df_invoices.head(100)

## Merge df_invoice and df_claims.

In [ ]:
df_invoice_claim = pd.merge(df_claims, df_invoices, on="invoice_id", how="left")

# Display summary information of df_invoice_claim
df_invoice_claim.info()
df_invoice_claim.head(100)

In [ ]:

# Mapping users' disability type:

df = pd.merge(df_invoice_claim, df_user, on="member_id", how="left")

# Make a copy for the clustering model. 
df_for_cluster = df.copy()

# Summary information for df 
df.head(100)

In [ ]:
#Groupby member_id 

df1 = df.groupby(["membership_number"]).agg({'invoice_total': 'sum', 'funded_total': 'sum'}).reset_index()

df1


In [ ]:
#subtract the invoice_total to fudned_total to see how user get reimbursemente - creat subtraction column

df1['subtraction'] = df1['invoice_total'] - df1['funded_total']
df1

In [ ]:
# Merging again

df2 = pd.merge(df1, df_user, on="membership_number", how="left")
df2.info()


In [ ]:
df2.head(20)

In [ ]:
#Select interested columns

cols_of_interest = {'membership_number', 'invoice_total', 'funded_total', 'subtraction', 'status', 'price_zone_code', 
                   'u_disabilities', 'u_gender'}
df2 = df2[cols_of_interest]
df2.info()

In [ ]:
#Re-arrange that mess

df2 = df2[['membership_number', 'invoice_total', 'funded_total', 'subtraction',   
                   'u_disabilities', 'u_gender','price_zone_code','status']]

df2.head(20)

In [ ]:
#Checking u_dis

df2['u_disabilities'].value_counts()

In [ ]:
# Replace blank value with 'others'

df2['u_disabilities'] = df2['u_disabilities'].replace([''],'others')

df2['u_disabilities'].value_counts()

In [ ]:
# Aggregate all categories != others into Intellectual

df2['u_disabilities'].mask(df2['u_disabilities'] != 'others', "Intellectual", inplace = True)

In [ ]:
df2['u_disabilities'].value_counts()

In [ ]:
df2.info()

In [ ]:
#Exploring to have a clearer view on the overall of reimbursement:

sub_0 = df2[df2['subtraction']> 1000]
#sub_0

sub_1 = df2[df2['subtraction']> 10000]
sub_1

#Set value for rows matching condition

#df2[['subtraction'] > 1000] == 'High'
#df2


In [ ]:
# Modifications to df2
cleaned_df2 = df2

# gender		: Add "other" for blank or null values --> Male/Female/Other/Unknown
cleaned_df2["u_gender"] = cleaned_df2["u_gender"].replace([""],"Unknown")

In [ ]:
# status		: drop
cleaned_df2 = cleaned_df2.drop(columns=["status"])

In [ ]:
# price_code	: drop, replace with SA4 and its information
cleaned_df2 = cleaned_df2.drop(columns=["price_zone_code"])
df_user_min = df_user[["membership_number", "SA4"]]

In [ ]:
# Remove rows with blank SA1 - SA4 values
blankIndices = df_user_min[df_user_min["SA4"] == 0.0].index
df_user_min = df_user_min.drop(axis=0, labels=blankIndices)

In [ ]:
# Drop rows with null SA1-4 values 
df_user_min = df_user_min.dropna()
# Drop rows with duplicate membership_number values 
df_user_min = df_user_min.drop_duplicates()

In [ ]:
# Obtain geographical locations using SA4
df_sa4 = pd.read_csv("./TableDump/SA4_2016.csv",sep='\t')
df_sa4

In [ ]:
#Split values into columns

df_sa4[['SA4_CODE_2016','SA4_NAME_2016','GCCSA_CODE_2016','GCCSA_NAME_2016','STATE_CODE_2016','STATE_NAME_2016','AREA_ALBERS_SQKM']] = df_sa4['SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM'].str.split(',',expand=True)
df_sa4

In [ ]:
# Extract relevant columns
df_sa4 = df_sa4[["SA4_CODE_2016", "SA4_NAME_2016", "GCCSA_NAME_2016", "STATE_NAME_2016"]]
# Rename columns to match df2
name_mapping = {
    "SA4_CODE_2016": "SA4",
    "SA4_NAME_2016": "SA4_NAME",
    "GCCSA_NAME_2016": "GCCSA_NAME",
    "STATE_NAME_2016": "STATE_NAME"
}
df_sa4 = df_sa4.rename(columns=name_mapping)
df_sa4

In [ ]:
#Set float for SA4:
df_sa4['SA4'] = df_sa4['SA4'].astype(float, errors = 'raise')


In [ ]:
# Merge SA4 information with the user DF
df_user_min = pd.merge(df_user_min, df_sa4, on= "SA4", how="left")
# Merge the final user info with DF2
cleaned_df2 = pd.merge(cleaned_df2, df_user_min, on="membership_number", how="left")


In [ ]:
# Add Member_key to deal with invoice calculations
cleaned_df2 = pd.merge(cleaned_df2, df_user[["membership_number", "member_key"]], on="membership_number", how="left")
# Drop duplicated member keys
cleaned_df2 = cleaned_df2.drop_duplicates()

In [ ]:
# Display Summary information for cleaned_df2
cleaned_df2.info()
cleaned_df2.head(100)

In [ ]:
# Get Completed Plans
df_plan = pd.read_sql("select p.plan_key, p.member_key, p.status, p.start_date, p.end_date, pb.item_category_level2_key, pb.allocation, pb.remaining from HH_plan p join HH_plan_budget pb on p.plan_key = pb.plan_key where p.status = 'COMPLETED'", con=conn)
# Convert start and end_date to datetime
df_plan["start_date"] = pd.to_datetime(df_plan["start_date"], format="%Y-%m-%d")
df_plan["end_date"] = pd.to_datetime(df_plan["end_date"], format="%Y-%m-%d")
# Summary info for df_plan
df_plan.info()
df_plan.head(100)


In [ ]:
df_plan2 = df_plan

# Convert start and end_date to datetime
df_plan2["start_date"] = pd.to_datetime(df_plan2["start_date"], format="%Y-%m-%d")
df_plan2["end_date"] = pd.to_datetime(df_plan2["end_date"], format="%Y-%m-%d")


In [ ]:
# Sum up all based on plan_key

df_grouped_plan2 = df_plan2.groupby(["plan_key"]).agg({"allocation": "sum", "remaining": "sum", "member_key": "first", "start_date": "first", "end_date": "first"}).reset_index()

# Sort grouped df by start and end dates in descending order

df_grouped_plan2 = df_grouped_plan2.sort_values(["start_date", "end_date"], ascending=[False, False])

# Eliminate duplicate member keys by dropping all rows but the most recent ones

df_grouped_plan2 = df_grouped_plan2.groupby(["member_key"]).agg({"start_date": "first", "end_date": "first", "plan_key": "first", "allocation": "first", "remaining": "first"}).reset_index()


In [ ]:
# Summary information for df_plan2

df_grouped_plan2.info()
df_grouped_plan2.head(100)

In [ ]:
# Merge plan information with cleaned_df2

temp_df2 = pd.merge(cleaned_df2, df_grouped_plan2, on="member_key", how="left")

# Drop members without completed plans

temp_df2 = temp_df2.dropna(subset=["plan_key"])


In [ ]:
#Calculate ratio of spent-allocated

temp_df2["spending_ratio"] = (temp_df2["allocation"] - temp_df2["remaining"]) / temp_df2["allocation"]
temp_df2["spent"] = temp_df2["allocation"] - temp_df2["remaining"]

# Variables defining under and appropriate spending ratio thresholds.
                                                      
under_spend_thres = 0.75
par_spend_thres = 1.0

temp_df2["under_spent"] = temp_df2["spending_ratio"] <= under_spend_thres
temp_df2["over_spent"] = temp_df2["spending_ratio"] > par_spend_thres
temp_df2["par_spent"] = temp_df2["spending_ratio"] == par_spend_thres

In [ ]:
# Drop outdated columns

temp_df2 = temp_df2.drop(columns=["invoice_total", "funded_total"])

In [ ]:
# Display Summary information for temp_df2

temp_df2.info()
temp_df2.head(100)
temp_df2["under_spent"].value_counts()


In [ ]:
# One-Hot Encoding of temp_df2

one_hot_df2 = temp_df2.set_index("membership_number")

# One-hot encoding of Gender

one_hot_gender = pd.get_dummies(one_hot_df2["u_gender"], prefix="gender")

# Merge with the cleaned DF2

one_hot_df2 = pd.merge(one_hot_df2, one_hot_gender, on="membership_number", how="left")


In [ ]:
# One-hot encoding of GCCSA

# Compile list of Greater regions

greaterRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Greater", na=False)]["GCCSA_NAME"].value_counts().index.to_list()

# Compile list of "Rest of..." regions

restOfRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Rest of", na=False)]["GCCSA_NAME"].value_counts().index.to_list()

# Compile list of "Capital" regions

capitalRegions = one_hot_df2[one_hot_df2["GCCSA_NAME"].str.contains("Capital", na=False)]["GCCSA_NAME"].value_counts().index.to_list()

# Replace "Greater" values with Urban

for region in greaterRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Urban")

# Replace "Rest of..." values with Rural

for region in restOfRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Rural")
    
# Replace "Capital" regions with Urban

for region in capitalRegions:
    one_hot_df2 = one_hot_df2.replace([region], "Urban")
    
# Replace NaN values with "Unknown"

one_hot_df2["GCCSA_NAME"] = one_hot_df2["GCCSA_NAME"].replace(np.nan, "Unknown")

# Perform One-hot Encoding on GCCSA_NAME

one_hot_region = pd.get_dummies(one_hot_df2["GCCSA_NAME"], prefix="GCCSA")
one_hot_df2 = pd.merge(one_hot_df2, one_hot_region, on="membership_number", how="left")

In [ ]:
# Convert any UINT8 columns to Bool

column_names = one_hot_df2.select_dtypes(include=[np.uint8]).columns
one_hot_df2[column_names] = one_hot_df2[column_names].astype(bool)


In [ ]:
# Drop unnecessary columns

one_hot_df2 = one_hot_df2.drop(columns=["SA4", "SA4_NAME", "GCCSA_NAME", "STATE_NAME", "u_gender"])

In [ ]:
# Display Summary information for one_hot_df2

one_hot_df2.info()
one_hot_df2.head(100)

In [ ]:
# Manual one-hot encode subtraction:
# Assumed threshold for determining over/under-spending
funded_threshold = 500
df = one_hot_df2
df['subtraction'] = df['subtraction'].astype(int)
df['not_fully_funded'] = df['subtraction'].ge(funded_threshold)
df['acceptable_funded'] = df['subtraction'].lt(funded_threshold)
df


In [ ]:
#One hot encode u_disabilities
df = pd.get_dummies(df, prefix=['u_dis'], columns=['u_disabilities'])

## MODEL 1 - CLUSTERING

In [ ]:
#Pre processing for clustering
df_og = df_for_cluster
df_og.info()

In [ ]:
#for getting the avg of days from start to end
df_og['start_date'] = pd.to_datetime(df_og['start_date'])
df_og['funded_date'] = pd.to_datetime(df_og['funded_date'])
df_og['days_between_start_funded'] = (df_og['funded_date'] - df_og['start_date']).dt.days


In [ ]:
#Groupby member_id 

df_clus = df_og.groupby(["membership_number"]).agg({'days_between_start_funded': 'mean'}).reset_index()

df_clus = df_clus.dropna(subset=['days_between_start_funded'])
df_clus.info()
df_clus


In [ ]:
final_df1 = pd.merge(temp_df2, df_clus, on="membership_number", how="left")

final_df1.info()

## MODEL 2 - NEURAL NETWORK


In [ ]:

# Drop columns with unique values.
df = df.drop(columns=['spending_ratio',"plan_key", "member_key","start_date", "end_date", 'subtraction', 'allocation', 'remaining', 'spent'])
# Summary information for df
df.info()
df.head(100)


In [ ]:
# Check out model persistence where the trained model's weight was stored and loaded
# To uncomment this code if you wish to train the model again or any changes subjected to the dataset

# Specify the target column for analysis
target_column = "under_spent"
# Create the input df
input_df = df.drop([target_column], axis=1)
# Create a target df
target_df = df[target_column]
# Set random state
random_state = 10
# Set test size
test_size = 0.3
# Nump-ify input_df2
input_df_mat = input_df.to_numpy()
# Split training and test data
input_df_train, input_df_test, target_df_train, target_df_test = train_test_split(input_df_mat, target_df, test_size=test_size, stratify=target_df, random_state=random_state)
# Get standard scaler
scaler = StandardScaler()
# Transform training and test data
input_df_train = scaler.fit_transform(input_df_train, target_df_train)
input_df_test = scaler.transform(input_df_test)
# Generate a prediction
model_1 = MLPClassifier(random_state=random_state)
model_1.fit(input_df_train, target_df_train)
target_prediction = model_1.predict(input_df_test)



In [ ]:
# Summary Information
print("Classification Report: \n", classification_report(target_df_test, target_prediction))
print("Train Accuracy: ", model_1.score(input_df_train, target_df_train))
print("Test Accuracy: ", model_1.score(input_df_test, target_df_test))
print("Default Model Characteristics: ", model_1)

In [ ]:
# Improved Model - Neural Network tuned with GridSearchCV
hiddenLayerSizes = [(2,), (3, ), (4, ), (5, ), (6, ), (7, )]

alpha = [0.01, 0.001, 0.0001, 0.00001]

params = {'hidden_layer_sizes': hiddenLayerSizes, 'alpha': alpha}

model_2 = GridSearchCV(param_grid=params, estimator=MLPClassifier(random_state=random_state), cv=10, n_jobs=-1)

model_2.fit(input_df_train, target_df_train)

target_prediction = model_2.predict(input_df_test)
# Summary Information
print("Classification Report: \n", classification_report(target_df_test, target_prediction))
print("Train Accuracy: ", model_2.score(input_df_train, target_df_train))
print("Test Accuracy: ", model_2.score(input_df_test, target_df_test))
print("Tuned Model Characteristics: ", model_2)
print("Best Parameters: \n", model_2.best_params_)

### MODEL PERSISTENCE

In [ ]:
import pickle

#Save model weight:

filename = 'finalized_model.sav'
pickle.dump(model_1, open(filename, 'wb'))


In [ ]:
# some time later...
 
# load the model from disk

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(input_df_test, target_df_test)
print(result)

In [ ]:
'''
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = [[33,2,0,0,0,0,0,0,0,1,3], 
        [3,31,0,0,0,0,0,0,0,0,0], 
        [0,4,41,0,0,0,0,0,0,0,1], 
        [0,1,0,30,0,6,0,0,0,0,1], 
        [0,0,0,0,38,10,0,0,0,0,0], 
        [0,0,0,3,1,39,0,0,0,0,4], 
        [0,2,2,0,4,1,31,0,0,0,2],
        [0,1,0,0,0,0,0,36,0,2,0], 
        [0,0,0,0,0,0,1,5,37,5,1], 
        [3,0,0,0,0,0,0,0,0,39,0], 
        [0,0,0,0,0,0,0,0,0,0,38]]
df_cm = pd.DataFrame(array, index = [i for i in "ABCDEFGHIJK"],
                  columns = [i for i in "ABCDEFGHIJK"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

'''